# Tobig's 21기 2주차 Optimization 과제

# Gradient Descent 구현하기

### 1)"..."표시되어 있는 빈 칸을 채워주세요
### 2)강의내용과 코드에 대해 공부한 내용을 마크다운 또는 주석으로 설명해주세요

## 데이터

In [31]:
import pandas as pd
import numpy as np
import random

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
data = pd.read_csv("/content/drive/MyDrive/week2/assignment_2.csv")
data.head()

,Label,bias,experience,salary
0,1,1,0.7,48000
1,0,1,1.9,48000
2,1,1,2.5,60000
3,0,1,4.2,63000
4,0,1,6.0,76000


## data split

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size = 0.25, random_state = 0)

In [41]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((150, 3), (50, 3), (150,), (50,))

## Scaling

experience와 salary의 단위, 평균, 분산이 크게 차이나므로 scaler를 사용해 단위를 맞춰줍니다.

In [42]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
bias_train = X_train["bias"]
bias_train = bias_train.reset_index()["bias"]
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_train["bias"] = bias_train
X_train.head()

,bias,experience,salary
0,1,0.187893,-1.143335
1,1,1.185555,0.043974
2,1,-0.310938,-0.351795
3,1,-1.629277,-1.341220
4,1,-1.308600,0.043974


이때 scaler는 X_train에 fit 해주시고, fit한 scaler를 X_test에 적용시켜줍니다.  
똑같이 X_test에다 fit하면 안돼요!

In [43]:
bias_test = X_test["bias"]
bias_test = bias_test.reset_index()["bias"]
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
X_test["bias"] = bias_test
X_test.head()

,bias,experience,salary
0,1,-1.344231,-0.615642
1,1,0.508570,0.307821
2,1,-0.310938,0.571667
3,1,1.363709,1.956862
4,1,-0.987923,-0.747565


In [44]:
# parameter 개수
N = len(X_train.loc[0])

In [45]:
# 초기 parameter들을 임의로 설정해줍니다.
parameters = np.array([random.random() for i in range(N)])
random_parameters = parameters.copy()
parameters

array([0.69892257, 0.05798763, 0.23016761])

### * LaTeX   

Jupyter Notebook은 LaTeX 문법으로 수식 입력을 지원하고 있습니다.  
LaTeX문법으로 아래의 수식을 완성해주세요  
http://triki.net/apps/3466  
https://jjycjnmath.tistory.com/117

## Dot product
## $z = X \cdot parameters$

In [46]:
def dot_product(X, parameters):
    z = np.dot(X, parameters)
    return z

## Logistic Function

## $p = {1 \over 1 + e^{-x}},~x = x_1\theta_1 + \cdots + x_n\theta_n$

In [47]:
def logistic(X, parameters):
    x = np.dot(X, parameters)
    p = 1 / (1 + np.exp(x * -1))
    return p

In [48]:
logistic(X_train.iloc[1], parameters)

0.6852039577632227

## Object function

Object Function : 목적함수는 Gradient Descent를 통해 최적화 하고자 하는 함수입니다.  
<br>
선형 회귀의 목적함수
## $l(\theta) = \frac{1}{2}\Sigma(y_i - \theta^{T}X_i)^2$  
참고) $\hat{y_i} = \theta^{T}X_i$. 1/2 은 계산상의 편의를 위해 곱합니다
  
로지스틱 회귀의 목적함수를 작성해주세요  
(선형 회귀의 목적함수처럼 sum 형태까지만 작성해주세요. 평균을 고려하는 것은 뒤에 코드에서 수행합니다)
## $l(p) = \sum^n_{i=1} [y_i log(p(x_i)) + (1 - y_i)log(1-p(x_i))]$

In [49]:
def minus_log_cross_entropy_i(X, y, parameters):
    loss = y * logistic(X, parameters) + (1 - y) * logistic(1-X, parameters)
    return loss

In [50]:
def mse_i(X, y, parameters):
    loss = (y - np.dot(X, parameters)) ** 2
    return loss

In [51]:
def batch_loss(X_set, y_set, parameters, loss_function, n): #n: 현재 배치의 데이터 수
    loss = 0

    for i in range(n):
        loss += loss_function(X_set.iloc[i], y_set.iloc[i], parameters)

    loss = loss/n #loss 평균값으로 계산
    return loss

In [52]:
batch_loss(X_test, y_test, parameters, mse_i, len(X_test))

0.6183774861211611

In [53]:
batch_loss(X_test, y_test, parameters, minus_log_cross_entropy_i, len(X_test))

0.570898910168542

## Gradient
위의 선형회귀의 목적함수 $l(\theta)$와 로지스틱회귀의 목적함수 $l(p)$의 gradient를 작성해주세요  
(위의 목적함수를 참고해서 작성해주세요 = 평균을 고려하는 것은 뒤에 코드에서 수행합니다)

## ${\partial\over{\partial \theta_j}}l(\theta)=\sum^n_{i=1} (y_i - \theta^T X_i) \cdot (-X_i)$
## ${\partial\over{\partial \theta_j}}l(p)=\sum^n_{i=1} (y_i−p(X_i))$

In [54]:
def get_gradient_ij(X, y, parameters, j, model):
    if model == 'linear':
        gradient = (y - np.dot(parameters, X.iloc[j])) * (-1 * X.iloc[j])
    else:
        gradient = y - logistic(X.iloc[j], parameters)
    return gradient

In [55]:
get_gradient_ij(X_train.iloc[0,:], y_train.iloc[0], parameters, 1, 'linear')

array([-0.16321842, -0.18584586, -0.17976725])

In [56]:
get_gradient_ij(X_train.iloc[0,:], y_train.iloc[0], parameters, 1, 'logistic')

array([0.46721643, 0.49727616, 0.48918996])

In [57]:
X_train.iloc[0,:]

bias          1.000000
experience    0.187893
salary       -1.143335
Name: 0, dtype: float64

In [68]:
# from IPython.display import Image

# Image("C:/Users/rhskr/Desktop/배치알고리즘_구현.png")

!wget -O "배치알고리즘_구현.png" "C:/Users/rhskr/Desktop/배치알고리즘_구현.png"

--2024-01-30 13:53:38--  ftp://c//Users/rhskr/Desktop/%EB%B0%B0%EC%B9%98%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_%EA%B5%AC%ED%98%84.png
           => ‘배치알고리즘_구현.png’
Resolving c (c)... failed: Name or service not known.
wget: unable to resolve host address ‘c’


## Batch Gradient
하나의 배치 (X_set, y_set)에 대해 기울기를 구하는 코드를 작성해주세요

In [69]:
def batch_gradient(X_set, y_set, parameters, model):
    gradients = [0 for _ in range(len(parameters))]

    for idx in range(len(X_set.iloc[0,:])):
        gradient = 0
        for i in range(len(y_set)):
            gradient += get_gradient_ij(X_set.iloc[i, :], y_set.iloc[i], parameters, idx, model)[idx]
        gradients[idx] = gradient/len(y_set)

    return gradients

In [70]:
gradients1 = batch_gradient(X_train, y_train, parameters, 'linear')
gradients1

[0.41894843265571147, -0.06254379310278407, 0.32384778634825573]

In [71]:
gradients1 = batch_gradient(X_train, y_train, parameters, 'logistic')
gradients1

[-0.38795458493061774, -0.21999936449508212, -0.21993324285521307]

## mini-batch
인덱스로 미니 배치 나누기

In [72]:
def batch_idx(X_train, batch_size):
    N = len(X_train)
    nb = (N // batch_size)+1 #number of batch
    idx = np.array([i for i in range(N)])
    idx_list = [idx[i*batch_size:(i+1)*batch_size] for i in range(nb) if len(idx[i*batch_size:(i+1)*batch_size]) != 0]
    return idx_list

batch_idx 함수에 대한 설명을 batch_size와 함께 간략하게 작성해주세요  
### 설명:
우선 데이터 셋의 전체 크기와 지정한 배치 사이즈를 통해 전체 배치 개수를 구한다  
그 후 인덱스를 기준으로 배치에 해당하는 인덱스를 추출  
최종적으로 배치에 해당하는 인덱스를 반환  

## Update Parameters
기울기를 갱신하는 코드를 작성해주세요  
(loss와 마찬가지로 기울기를 갱신할 때 배치 사이즈를 고려해 평균으로 갱신해주세요)

In [73]:
def step(parameters, gradients, learning_rate, n): #n: 현재 배치의 데이터 수
    parameters -= [learning_rate * gradient / n for gradient in gradients]
    return parameters

In [74]:
step(parameters, gradients1, 0.01, len(X_train))

array([0.6989743 , 0.05801697, 0.23019693])

## Gradient Descent
위에서 작성한 함수들을 조합해서 경사하강법 함수를 완성해주세요

- learning_rate: 학습률, 한번에 얼마나 기울기를 감소 혹은 증가시킬 지를 정하는 것  
- tolerance: Step이 너무 작아서 더 이상의 학습이 무의미할 때 학습을 멈추는 조건    
- batch: 배치 사이즈를 의미   
- epoch: 전체 데이터를 모두 1번 학습 하였을 때 1 epoch이 학습되었다고 한다  
- num_epoch: 총 몇 번의 epoch를 수행할 지 정해주는 조건  
<br>

BGD: 학습 1번에 전체 데이터셋을 모두 사용하는 방법  
SGD: 학습 1번에 미니 배치 단위의 데이터를 사용하는 방법  
MGD: 학습 1번에 1개의 데이터만을 사용하는 방법  
<br>
batch_size에 따른 경사하강법의 종류를 적어주세요  
batch_size=1 -> MGD   
batch_size=k -> SGD  
batch_size=whole -> BGD  

In [75]:
def gradient_descent(X_train, y_train, learning_rate=0.1, num_epoch=1000, tolerance=0.00001, model='logistic', batch_size=16):
    stopper = False

    N = len(X_train.columns)  # 수정: 열의 개수로 설정
    parameters = np.random.rand(N)
    loss_function = minus_log_cross_entropy_i if model == 'logistic' else mse_i
    loss = 999
    batch_idx_list = batch_idx(X_train, batch_size)

    for epoch in range(num_epoch):
        if stopper:
            break
        for idx in batch_idx_list:
            X_batch = X_train.iloc[idx]
            y_batch = y_train.iloc[idx]

            # 그래디언트 계산
            gradients = np.zeros(N)
            new_loss = 0
            for i in range(len(X_batch)):
                loss_i, grad_i = loss_function(X_batch.iloc[i], y_batch.iloc[i], parameters)
                gradients += grad_i
                new_loss += loss_i

            new_loss /= len(X_batch)
            gradients /= len(X_batch)

            # 파라미터 업데이트
            parameters = step(parameters, gradients, learning_rate, len(X_batch))

            # 중단 조건
            if abs(new_loss - loss) < tolerance:
                stopper = True
                break
            loss = new_loss

        # 100epoch마다 학습 상태 출력
        if epoch % 100 == 0:
            print(f"epoch: {epoch}  loss: {new_loss}  params: {parameters}")

    return parameters


## Implement
경사하강법 함수를 이용해 최적의 모수를 찾아보세요. 학습을 진행할 때, hyperparameter를 바꿔가면서 학습시켜보세요.

## Logistic Regression

default: learning_rate = 0.1, num_epoch = 1000, tolerance = 0.00001, model = 'logistic', batch_size = 16

In [76]:
new_param_bgd = gradient_descent(X_train, y_train, batch_size=X_train.shape[0])
new_param_bgd

TypeError: cannot unpack non-iterable numpy.float64 object

In [77]:
new_param_sgd = gradient_descent(X_train, y_train, batch_size=1)
new_param_sgd

TypeError: cannot unpack non-iterable numpy.float64 object

In [78]:
new_param_mgd = gradient_descent(X_train, y_train)
new_param_mgd

TypeError: cannot unpack non-iterable numpy.float64 object

### Predict Label

In [79]:
# bgd 활용하여 학습한 parameters로 예측
y_predict = []
for i in range(len(y_test)):
    p = logistic(X_test.iloc[i,:], new_param_bgd)
    if p> 0.5 :
        y_predict.append(1)
    else :
        y_predict.append(0)

# 초기에 설정한 random parameters로 예측
y_predict_random = []
for i in range(len(y_test)):
    p = logistic(X_test.iloc[i,:], random_parameters)
    if p> 0.5 :
        y_predict_random.append(1)
    else :
        y_predict_random.append(0)

NameError: name 'new_param_bgd' is not defined

### Confusion Matrix

In [80]:
from sklearn.metrics import *

In [81]:
tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
confusion_matrix(y_test, y_predict)

ValueError: Found input variables with inconsistent numbers of samples: [50, 0]

In [82]:
accuracy = (tp+tn) / (tp+fn+fp+tn)
print("accuracy:",accuracy)

NameError: name 'tp' is not defined

## Linear regression
### $y = 0.5 + 2.7x$

### Data

In [83]:
raw_X = np.random.rand(150)
y = 2.7*raw_X + 0.5 + np.random.randn(150)

In [ ]:
tmp = np.array([1 for _ in range(150)])
X = np.vstack((tmp, raw_X)).T
X = pd.DataFrame(X)
y = pd.Series(y)

### Estimation

In [ ]:
#정규방정식
theta = np.linalg.inv(np.dot(X.T,X)).dot(X.T).dot(y)
theta

In [ ]:
#경사하강법
new_param = gradient_descent(X, y, model = 'linear')
new_param

In [ ]:
y_hat_NE = theta.dot(X.T)
y_hat_GD = new_param.dot(X.T)

### Visualization
시각화를 통해 정규방정식과 경사하강법을 통한 선형회귀를 비교해보세요  
(밑의 코드를 실행만 시키면 됩니다. 추가 코드 x)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(X.iloc[:,1], y, '.k') #산점도
plt.plot(X.iloc[:,1], y_hat_NE, '-b', label = 'NE') #정규방정식
plt.plot(X.iloc[:,1], y_hat_GD, '-r', label = 'GD') #경사하강법
plt.legend()
plt.show()